In [5]:
import numpy as np
import pandas as pd
import re, string

#import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from textblob import TextBlob

from scipy.sparse import coo_matrix

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.cluster import KMeans

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

from joblib import dump, load
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# --- Parser for reading in the Amazon json files (can be used for both reviews and metadata)
# --- credits folloing parse() method to Julian McAuley UCSD: http://jmcauley.ucsd.edu/data/amazon/ 
def parse(path):
    g = open(path, 'r')
    for l in g:
        yield json.loads(l)

In [2]:
ratings_matrix_file = "../data/amzn/ratings_matrix.csv"
book_ids_file = "../data/amzn/ratings_matrix_bookIDs.json"
user_ids_file = "../data/amzn/ratings_matrix_reviwerIDs.json"

books_ASIN = json.loads(open(book_ids_file).read()) #asin to idx lookup
reviewer_ID = json.loads(open(user_ids_file).read()) #reviewerID to idx lookup

print(f"Number of books with reviews: {len(books_ASIN)}")
print(f"Number of users: {len(reviewer_ID)}")

Number of books with reviews: 15782
Number of users: 210808


In [6]:
# --- Get ratings matrix (in book-user format) but convert to user-item format in COO matrix

ratings_matrix = []
with open(ratings_matrix_file) as f:
    for line in f:
        temp = line.strip().split(',')
        ratings_matrix.append([int(temp[0]), int(temp[1]), float(temp[2])])

ratings_matrix = np.array(ratings_matrix)
books = ratings_matrix[:,0].astype(int)
users = ratings_matrix[:,1].astype(int)
ratings = ratings_matrix[:,2].astype(float)

user_ratings = coo_matrix((ratings, (users, books)))
print(type(user_ratings))

<class 'scipy.sparse.coo.coo_matrix'>


In [25]:
#model = NMF(n_components=12, random_state=333)
#nmf = model.fit_transform(user_ratings)

In [7]:
# We'll use the famous SVD algorithm.
algo = SVD(verbose=True)

# you can also build KNNBasic and other types of models

# Run 5-fold cross-validation and print results
cross_validate(algo, user_ratings, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

# ml-100k dataset: this takes around .5 minute
# jester dataset: this takes around 10 minutes

AttributeError: raw_ratings not found